<a href="https://colab.research.google.com/github/kevalsakhiya/Data-Cleaning-and-Data-Analysis/blob/main/Data_cleaning_on_phone_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [12]:
phone_df = pd.read_csv('/content/drive/MyDrive/dataset/phone_data.csv')

In [13]:
phone_df.head(2)

,Unnamed: 0,model_url,model_name,price,rating,voting_count,general_sim_type,general_device_type,display_type,display_size,...,design_weight,display_ppi,connectivity_volte,technical_chipset,connectivity_5g,extra_water_resistance,battery_wireless_charging,battery_reverse_charging,battery_reverse_wireless_charging,display_foldable_display
0,0,https://www.smartprix.com/mobiles/intex-turbo-...,Intex Turbo i6,₹899,--rating:3;,5 votes,"Dual Sim, GSM+GSM",Feature Phone,Color TFT Screen (65K colors Colors),320 x 480 pixels,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,https://www.smartprix.com/mobiles/samsung-m880...,Samsung M8800 Pixon,"₹22,500",--rating:4.3;,96 votes,"Single Sim, GSM",NaN,Color TFT Screen (256K colors Colors),"3.2 inches, 240 x 400 pixels",...,121 g,~ 146 PPI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Issues with the dataset
---
1.   **Dirty Data (Quality Issues)**
---
1. **Price** -> It has Rupee sign(₹) and ',' in the data `Validity`
2. **model_name** -> Sometimes, the specs and years are included with the model name.`Validity`
    - ***Data Example*** -  Samsung Galaxy A12 Exynos 850 (6GB RAM + 128GB), Asus Zenfone Max ZC550KL 2016 (3GB RAM+32GB), Apple iPhone 5C (32GB) (Green, Blue, Yellow, Pink and White)
    - ***Row Numbers*** - 12359,12360,12361,12363
3. **rating** -> It has extra text beside the data.`Validity`
    - ***Data Example*** - --rating:3; , --rating:3.3;
4. **voting_count** -> It has extra text (votes) after the number of votes and ',' between the number.`Validity`
    - ***Data Example*** - 96 votes, 26,359 votes
5. **general_sim_type** -> It has multiple types of data combined: 1. the number of SIM cards supported, 2. the SIM card type, and 3. the SIM card slot type.`Validity`
    - ***Data Example*** - Single Sim, GSM , Dual Sim, GSM+GSM, Dual Sim, GSM+GSM (Hybrid Slot)
6. **general_device_type** -> It has 5 categories in the data:- Smartphone, Feature Phone, iPod, Gaming Phone, Tablet. We Only want two of them, a Smartphone and a Feature Phone.`Validity`
    - It has space after text.
7. **display_type** -> It has supported colour count present in the data (eg. (16M Colors)) and the same type written with a different format making duplicate type `Consistency`
    - ***Data Example*** - Color TFT Screen (65K colors Colors), Color IPS Screen, IPS LCD Screen, Color TFT Screen (256K Colors)
8. **display_size** -> There are 2 types of data available: Size in pixels and size in Inches. Some rows contain individual data, and some contain multiple data combined. `Validity`
    - \u2009 (Thin space) is present in the data.
    - ***Data Example*** - 320 x 480 pixels,4 inches, 3.2 inches, 240 x 400 pixels
9. **display_aspect_ratio** -> Some rows have extra space and tabs at the data's start, between and end. `Validity`
    - ***Data Example*** - \t19.5:9, '20.4:9 ', 20.5 : 9
    - ***Row Numbers*** - 2983, 12086, 4847, 10742, 14609, 7602
10. **memory_ram** -> The data has different Units, MB and GB, and \u2009 (Thin space) is present between value and unit. `Validity`
    - ***Data Example***- 5 MB, 256 MB, 8 GB, 2 GB
11. **memory_storage** -> The data has 4 different Units (KB, MB, GB, TB) and the presence of \u2009 between value and unit.`Validity`
    - Row number 10917 has NA GB and missing data in 3098 rows. `Accuracy`
    - ***Data Example***- 256 GB,512 KB, 50 MB, 64 GB, 16 TB
    - ***Row Numbers*** - 193, 1, 2, 2989
12. **memory_card_slot** -> It has mainly two types of data combined: 1. memory-card supported or not, 2. memory-card storage up to and sometimes there are also details about the card slot type. `Validity`
    * \u2009 and Multiple Units present in data (MB, GB, TB)
    - ***Data Example***- Yes, (Hybrid Slot), up to 1 TB, Yes, upto 8 GB, Yes, No, Yes, (Hybrid Slot), 858      Yes, upto 2 GB, 1 GB included,
    - ***Row Numbers*** - 11625, 4802, 14411, 16258, 2336, 10522, 858
13. **connectivity_3g** -> Some rows have information about download and upload speed, and some only have Yes or No. `Validity`
    - ***Data Example***- Yes 21 Mbps Download 5.76 Mbps Upload, Yes 1 Mbps Download, Yes, No
    - ***Row Numbers*** - 13935, 16133, 4810, 16003, 3801, 4012
14. **connectivity_wifi** -> Most rows have just Yes or No, but few have information about wifi-hotspot. `Validity`
    - ***Data Example***- Yes, No, Yes, with wifi-hotspot
    - ***Row Numbers*** - 3544, 3585, 112, 9088
15. **connectivity_bluetooth** -> It has data on Bluetooth availability and the version of Bluetooth with some extra information.`Validity`
    - ***Data Example***- Yes, Yes, v2.1, Yes, v5.0, A2DP, LE, Yes, v5.4, AAC/AptX/AptX HD/AptX Adaptive/LDAC/LHDC 5.0
    - ***Row Numbers*** - 7478, 718, 14717, 8207,10511
16. **connectivity_usb** -> Most of the data contains the USB's availability and type, and a few also have its version.`Validity`
    - ***Data Example***- Yes, No, Yes, microUSB v2.0, Yes, microUSB, Yes, USB-C v2.0, No, microUSB
    - ***Row Numbers*** - 2989,8580, 4614, 13582, 1209,1306
17. **extra_3.5mm_headphone_jack** -> Most of the data contains Yes or No, but some row has alternatives if there is no audio jack, and some have the sound quality supported, etc.`Validity`
    - ***Data Example***- No, USB-C to 3.5mm connector available in-box, No, Support standard Type-C earphone, Unspecified, Yes, Hi-Res Audio, 3.5
    - ***Row Numbers*** - 6443, 10537, 13325, 3914, 2085
18. **camera_rear_camera** -> Most of the rows contain information about the type of lenses and autofocus, as well as the Megapixels of all the cameras.`Validity`
    - \u2009 is present in the data.
    - ***Data Example***- 13 MP with autofocus, 48 MP + AI Portrait with autofocus, 50 MP f/1.8 (Main)\n24 MP f/1.8 (Night Vision)\n8 MP f/1.8 (Fusion)\n(Thermal Imaging Camera)
19. **camera_flash** -> It has two data types present: 1. Flash availability, and 2. The type of flash, and sometimes the only type of flash.`Validity`
    - ***Data Example*** - Yes, Six LED Dual Tone Flash, Yes, LED, LED
20. **camera_front_camera** -> It has the camera megapixels and type of camera, and it also includes information about the flash types.`Validity`
    -  \u2009 is present in the data.
    - ***Data Example*** - Notch 5 MP, Under Display 16 MP 26mm, 1/2.8", 1.22µm, under display f/2 (Wide Angle), 5 MP f/2.2 with Screen Flash
21. **technical_cpu** -> Most of the data contains a number of the cores of the CPU and its cycle speed, but some don't have core numbers.`Validity`
    -  cycle speed has three units, KHz, GHz and MHz
    -  \u2009 is present in the data.
    - ***Data Example*** - 1.25 GHz, Quad Core Processor, Single Core Processor, 1 MHz Processor.
22. **battery_size** -> This field contains information about battery size in mAh and battery material. Some rows don't have battery material.`Validity`
    - The battery type is written differently, like Li-ion Battery, Li-ion, Lion, Li-Po Battery, or Li-Polymer battery.`Consistency`
    - \u2009 is present in the data.`Validity`
    - ***Data Example*** - 4000 mAh, 5050 mAh, Li-Po Battery, 950 mAh, Li-ion Battery
23. **design_dimensions** ->  Most of the data has dimensions in mm, but a few also have inches.`Consistency`
    - \u2009 is present in the data.`Validity`
    - ***Data Example*** - 162.2 x 74.6 x 8.3 mm (6.39 x 2.94 x 0.33 in), 71.5 x 142 x 8.94 mm
24. **connectivity_volte** -> It has two types of data: 1. volte-connectivity and type of connecctivity.`Validity`
    - ***Data Example*** - Yes, Yes, Dual Stand-By, Dual Stand-By
25. **technical_chipset** - It combines the manufacturer, series and model of the chipset.`Validity`
    - Sometimes, only manufacturers and sometimes only models are available in the data.`Validity`
    - ***Data Example*** - Qualcomm Snapdragon 660, Spreadtrum SC7731G, Spreadtrum, BCM23550
26. **extra_water_resistance** -> It has ip_rating, and water resistance time combined with the data, and sometimes only ip_rating are in it.`Validity`
    - \u2009 is present in the data. `Validity`
    - ***Data Example*** - Yes, 6 m upto 30 min| IP68, IP67, IP54
27. **battery_wireless_charging** ->  It has a combination of data on wireless charging capability, supported power Watts, and sometimes the type of wireless charging.`Validity`
    - ***Data Example*** - Yes, Works with Qi Chargers| Yes, 50W Wireless Flash Charge| Yes, 15W
28. **battery_reverse_charging** - The capability of reverse charging and supported power Watts exist in the data.`Validity`
    - ***Data Example*** - Yes, 9W Reverse Charging | Yes, 10W | Yes, Reverse wireless charging

---
2.   **Messy Data (Tidiness Issues)**
---
1. **model_name** -> It Can be split into two columns: brand and model
2. **camera_rear_camera** -> It can be split into multiple columns containing the data for each lens and lens features.
3. **technical_cpu** -> It can be split into two columns containing the CPU's number of cores and clock cycles.
4. **battery_size** -> It can be split into columns containing data on battery size and material.
5. **technical_chipset** -> It can be split into the manufacturer, series and model of the chipset.


In [14]:
df = phone_df.copy()
df.drop(columns=['Unnamed: 0'],inplace=True)

In [15]:
df.columns

Index(['model_url', 'model_name', 'price', 'rating', 'voting_count',
       'general_sim_type', 'general_device_type', 'display_type',
       'display_size', 'display_aspect_ratio', 'memory_ram', 'memory_storage',
       'memory_card_slot', 'connectivity_3g', 'connectivity_4g',
       'connectivity_wifi', 'connectivity_bluetooth', 'connectivity_usb',
       'extra_3.5mm_headphone_jack', 'camera_rear_camera', 'camera_flash',
       'camera_front_camera', 'technical_cpu', 'battery_type', 'battery_size',
       'design_dimensions', 'design_weight', 'display_ppi',
       'connectivity_volte', 'technical_chipset', 'connectivity_5g',
       'extra_water_resistance', 'battery_wireless_charging',
       'battery_reverse_charging', 'battery_reverse_wireless_charging',
       'display_foldable_display'],
      dtype='object')

## model_name

In [16]:
# Issue - Sometimes, the specs and years are included with the model name

df['model_name'].value_counts().sample(3)

model_name
Spice Boss Power M-5374    1
Trio Junior 3              1
Karbonn K9 Spy             1
Name: count, dtype: int64

In [17]:
def split_brand_model(model_name):
    '''
    Separate brand and model and returns after cleaning it.
    '''
    brand_model_split = model_name.split('(')[0].split(' ', 1)
    brand = brand_model_split[0].capitalize()

    model_with_year = brand_model_split[1].capitalize() if len(brand_model_split) > 1 else ''
    # cleans the data if model name has year or any other data like - Asus Zenfone Max ZC550KL 2016 (3GB RAM+32GB)
    model = re.sub(r'\b\d{4}\b', '', model_with_year).strip()
    return pd.Series([brand, model])

In [18]:

brand_model = df['model_name'].apply(split_brand_model)
brand = brand_model[0]
model = brand_model[1]
# Inserting the column after model_name column
df.insert(2, 'brand', brand)
df.insert(3, 'model', model)

## Price

In [19]:
# Issue - It has Rupee sign(₹) and ',' in the data

df['price'].sample(3)

7152     ₹25,990
3368      ₹7,999
10965    ₹25,999
Name: price, dtype: object

In [20]:

price = df['price'].str.replace('₹','').str.replace(',','').astype('int')
# getting the position of the price
df.columns.get_loc('price')

4

In [21]:
# inserting after price column
df.insert(3, 'price_INR', price)

In [22]:
df['price_INR'].describe()

count     17153.000000
mean      11819.863989
std       20584.361762
min          99.000000
25%        1299.000000
50%        4495.000000
75%       13999.000000
max      489990.000000
Name: price_INR, dtype: float64

## Rating

In [23]:
# Issue - It has extra text beside the data.

df['rating'] = df['rating'].str.replace('--rating:','').str.replace(';','').astype(float)

## voting_count

In [24]:
# Issue - It has extra text (votes) after the number of votes and ',' between the number

df['voting_count'] = df['voting_count'].str.replace('votes','').str.replace(',','').str.strip().astype('int')

## general_sim_type

In [25]:
# Issue - It has multiple types of data combined: 1. the number of SIM cards supported, 2. the SIM card type, and 3. the SIM card slot type

# Removing sim types as most of them are GSM and only 0.9% has CDMA supported.
sim_type = df['general_sim_type'].str.extract(r'([a-zA-Z]+ Sim)')
sim_type.value_counts()

Dual Sim      15716
Single Sim     1356
Triple Sim       50
Quad Sim         13
No Sim            1
Name: count, dtype: int64

In [26]:
df.columns.get_loc('general_sim_type')

8

In [27]:
sim_type = sim_type.astype('category')
# Inserting column after general_sim_type
df.insert(8, 'sim_type', sim_type)

## general_device_type

In [28]:
# Issue - It has 5 categories in the data:- Smartphone, Feature Phone, iPod, Gaming Phone, Tablet. We Only want two of them, a Smartphone and a Feature Phone.

df['general_device_type'].value_counts()

general_device_type
Smartphone       9960
Feature Phone    5817
iPod                1
Gaming Phone        1
Tablet              1
Name: count, dtype: int64

In [29]:
# Replacing smartphone with Smart Phone
df['general_device_type'] = df['general_device_type'].replace('Smartphone','Smart Phone')

In [30]:
# Filtering all rows that have either Smartphone or Feature Phone in general_device_type
df = df[df['general_device_type'].isin(['Smart Phone','Feature Phone'])]
df['general_device_type'].value_counts()

general_device_type
Smart Phone      9960
Feature Phone    5817
Name: count, dtype: int64

In [31]:
general_device_type = df['general_device_type'].astype('category')
# Using `df.columns.get_loc('general_device_type')+1` position next to general_device_type
df.insert(df.columns.get_loc('general_device_type')+1, 'device_type', general_device_type)

## display_type

In [32]:
# Issue - It has supported colour count present in the data (eg. (16M Colors)) and the same type written with a different format making duplicate type

df['display_type'].value_counts()

display_type
Color TFT Screen                         3020
Color                                    1501
Color IPS LCD Screen (16M Colors)        1435
Color IPS Screen                          995
Color IPS Screen (16M colors Colors)      968
                                         ... 
Color TFT Liquid Crystal (16M Colors)       1
Color OLED Screen (1.07M)                   1
Color Pol-less Plus OLED (1B)               1
Color Dynamic AMOLED 2X, (1B)               1
TFT Screen (Black)                          1
Name: count, Length: 272, dtype: int64

In [33]:
def categorize_display_types(display_type):
    '''
    Categorizing common the display type out of given input.
    '''
    display_type = display_type.lower()

    if 'ips' in display_type:
        return 'IPS LCD'
    elif 'tft' in display_type:
        return 'TFT LCD'
    elif 'amoled' in display_type:
        return 'AMOLED'
    elif 'oled' in display_type:
        return 'OLED'
    else:
        return 'Other'

In [34]:
# removing extra data from column
df['display_type'] = df['display_type'].fillna('other').str.split('(').str.get(0)

# Applying the categorize function
display_type = df['display_type'].apply(categorize_display_types)

# Changing column type to category
df['display_type'] = display_type.astype('category')
df['display_type'].value_counts()

<ipython-input-34-e7856fd6373c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['display_type'] = df['display_type'].fillna('other').str.split('(').str.get(0)
<ipython-input-34-e7856fd6373c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['display_type'] = display_type.astype('category')


display_type
IPS LCD    4544
TFT LCD    4428
Other      4218
AMOLED     2057
OLED        530
Name: count, dtype: int64

## display_size

In [35]:
# Issue - There are 2 types of data available: Size in pixels and size in Inches. Some rows contain individual data, and some contain multiple data combined.

# Extracting data
df['display_size'] = df['display_size'].str.replace('\u2009',' ')
display_pixel_count = df['display_size'].str.extract(r'([\d]+\s*x\s*[\d]+) pixels')
display_size_inches = df['display_size'].str.extract(r'([\d.]+) inches').astype('float')

In [36]:
# inserting columns
df.insert(df.columns.get_loc('display_size')+1,'display_pixel_count',display_pixel_count)
df.insert(df.columns.get_loc('display_size')+2,'display_size_inches',display_size_inches)

## memory_ram

In [37]:
# Issue - The data has different Units, MB and GB, and \u2009 (Thin space) is present between value and unit.

def convert_to_gb(memory_storage):
    if memory_storage:
        try:
            memory_storage = memory_storage.lower().strip()
            if 'mb' in memory_storage:
                memory_mb = float(memory_storage.replace('mb', '').strip())
                memory_gb = memory_mb / 1024
                return memory_gb
            elif 'gb' in memory_storage:
                memory_gb = float(memory_storage.replace('gb', '').strip())
                return memory_gb
            elif 'tb' in memory_storage:
                memory_tb = float(memory_storage.replace('tb', '').strip())
                memory_gb = memory_tb * 1024
                return memory_gb
            else:
                return 0
        except Exception as e:
            # print(memory_storage,'=>',e)
            return 0

In [38]:
# Converting all value in gb
memory_ram_gb = df['memory_ram'].fillna('0').apply(convert_to_gb).astype('float')
memory_ram_gb
df.insert(df.columns.get_loc('memory_ram'),'memory_ram_gb',memory_ram_gb)

## memory_storage

In [39]:
# The data has 4 different Units (KB, MB, GB, TB) and the presence of \u2009 between value and unit
memory_storage_gb = df['memory_storage'].fillna('0').apply(convert_to_gb).astype('float')
df.insert(df.columns.get_loc('memory_storage')+1,'memory_storage_gb',memory_storage_gb)

## memory_card_slot

In [40]:
# It has mainly two types of data combined: 1. memory-card supported or not, 2. memory-card storage up to and sometimes there are also details about the card slot type.

df['memory_card_slot'] = df['memory_card_slot'].str.replace('\u2009',' ')
has_memory_card_slot = df['memory_card_slot'].apply(lambda x: True if x=='Yes' else False).astype('category')
# has_memory_card_slot
memory_card_slot_capacity = df['memory_card_slot'].str.extract('(\d+(\.\d+)?\s*(GB|MB|TB))',expand=False)[0]
memory_card_slot_capacity = memory_card_slot_capacity.apply(convert_to_gb)

df.insert(df.columns.get_loc('memory_card_slot')+1,'has_memory_card_slot',has_memory_card_slot)
df.insert(df.columns.get_loc('memory_card_slot')+2,'memory_card_slot_capacity',memory_card_slot_capacity)


In [41]:
df.iloc[:2,13:]

,display_size,display_pixel_count,display_size_inches,display_aspect_ratio,memory_ram_gb,memory_ram,memory_storage,memory_storage_gb,memory_card_slot,has_memory_card_slot,...,design_weight,display_ppi,connectivity_volte,technical_chipset,connectivity_5g,extra_water_resistance,battery_wireless_charging,battery_reverse_charging,battery_reverse_wireless_charging,display_foldable_display
0,320 x 480 pixels,320 x 480,NaN,3:2,0.120117,123 MB,56 MB,0.054688,"Yes, upto 32 GB",False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"4.5 inches, 480 x 854 pixels",480 x 854,4.5,16:9,1.000000,1 GB,8 GB,8.000000,"Yes, upto 128 GB",False,...,138 g,~ 218 PPI,Yes,Qualcomm Snapdragon MSM8909,NaN,NaN,NaN,NaN,NaN,NaN


## connectivity_edge,3g,4g,wifi

In [42]:
# Issue -3g - Some rows have information about download and upload speed, and some only have Yes or No
# Issue- Wifi- Most rows have just Yes or No, but few have information about wifi-hotspot
has_3g = df['connectivity_3g'].fillna('').apply(lambda x: True if 'Yes' in x else False)
has_4g = df['connectivity_4g'].fillna('').apply(lambda x: True if 'Yes' in x else False)
has_wifi = df['connectivity_wifi'].fillna('').apply(lambda x: True if 'Yes' in x else False)

df.insert(df.columns.get_loc('connectivity_3g')+1,'has_3g',has_3g)
df.insert(df.columns.get_loc('connectivity_4g')+1,'has_4g',has_4g)
df.insert(df.columns.get_loc('connectivity_wifi')+1,'has_wifi',has_wifi)

## connectivity_bluetooth

In [43]:
# Issue - ost of the data contains the USB's availability and type, and a few also have its version
df['connectivity_bluetooth'].value_counts()

connectivity_bluetooth
Yes                                                        4667
Yes, v4.0                                                  1628
Yes, v5.0                                                  1050
Yes, v2.0                                                   707
Yes, v4.2                                                   684
                                                           ... 
Yes, v40.0                                                    1
Yes, v5.4, AAC/AptX/AptX HD/AptX Adaptive/LDAC/LHDC 5.0       1
Yes, v4.2, A2DP, LE, EDR, aptX                                1
Yes, v5.4, SBC, AAC, aptX, aptX HD, and LDAC                  1
Yes, v4.2, A2DP, aptX, LE                                     1
Name: count, Length: 164, dtype: int64

In [44]:
blouetooth_version = df['connectivity_bluetooth'].str.extract(r'(?:v|, )([\d.]+)')[0].astype('float')
has_bluetooth = df['connectivity_bluetooth'].str.split(',').str.get(0).fillna('').apply(lambda x: True if 'Yes' in x else False).astype('category')
blouetooth_version.value_counts()
df.insert(df.columns.get_loc('connectivity_bluetooth')+1,'has_bluetooth',has_bluetooth)
df.insert(df.columns.get_loc('connectivity_bluetooth')+2,'bluetooth_version',blouetooth_version)

## connectivity_usb

In [45]:
# Issue - Most of the data contains Yes or No, but some row has alternatives if there is no audio jack, and some have the sound quality supported, etc
df['connectivity_usb'].value_counts()

connectivity_usb
Yes, microUSB v2.0                                                              3454
Yes                                                                             3152
Yes, USB-C v2.0                                                                 2470
Yes, microUSB                                                                   1647
Yes, USB-C                                                                       967
Yes, USB-C v3.1                                                                  347
No                                                                               228
Yes v2.0                                                                         174
Yes, USB-C v3.2                                                                  132
Yes, miniUSB                                                                     131
Yes, microUSB-MHL v2.0                                                            81
Yes, Proprietary v2.0                           

In [46]:
usb_version = df['connectivity_usb'].str.extract(r'(?<![A-Za-z])v?(\d+(\.\d+)?\'?)')[0].astype('float')
has_usb = df['connectivity_usb'].str.split(',').str.get(0).fillna('').apply(lambda x: True if 'Yes' in x else False).astype('category')
df.insert(df.columns.get_loc('connectivity_usb')+1,'has_usb',has_usb)
df.insert(df.columns.get_loc('connectivity_usb')+2,'usb_version',usb_version)

## extra_3.5mm_headphone_jack

In [47]:
# Issue - Most of the data contains Yes or No, but some row has alternatives if there is no audio jack, and some have the sound quality supported, etc
headphone_jack = df['extra_3.5mm_headphone_jack'].str.split(',').str.get(0).fillna('').apply(lambda x: True if x=='Yes' else False)
df.insert(df.columns.get_loc('connectivity_usb')+1,'has_3.5mm_audiojack',headphone_jack)

## camera_rear_camera

In [48]:
# Issue - Most of the rows contain information about the type of lenses and autofocus, as well as the Megapixels of all the cameras

df['camera_rear_camera'] = df['camera_rear_camera'].str.replace('\u2009',' ')

In [49]:
megapixels = df['camera_rear_camera'].str.extractall(r'([\d+.]+) MP').unstack().droplevel(0,axis=1)
megapixels.columns = [f'camera_{i+1}_mp' for i in megapixels.columns]

for i,col in enumerate(megapixels.columns,df.columns.get_loc('camera_rear_camera')+1):
    df.insert(i,col,megapixels[col])

In [50]:
def check_lens_types(camera_info):
    lens_types = ['Wide Angle', 'Ultra Wide', 'Telephoto', 'Depth Sensor', 'Night Vision']
    lens_type_info = {}

    for lens_type in lens_types:
        column_name = f'has_rear_camera_{lens_type.lower().replace(" ", "")}'
        lens_type_info[column_name] = lens_type.lower() in camera_info.lower()

    return lens_type_info

In [51]:
lens_type_df = df['camera_rear_camera'].fillna('').apply(check_lens_types)
lens_type_df = pd.json_normalize(lens_type_df)
lens_type_df.head(2)

,has_rear_camera_wideangle,has_rear_camera_ultrawide,has_rear_camera_telephoto,has_rear_camera_depthsensor,has_rear_camera_nightvision
0,False,False,False,False,False
1,False,False,False,False,False


In [52]:
# Insering above columns
for i,col in enumerate(lens_type_df.columns,df.columns.get_loc('camera_rear_camera')+1):
    df.insert(i,col,lens_type_df[col])

## camera_flash

In [53]:
# Issue - It has two data types present: 1. Flash availability, and 2. The type of flash, and sometimes the only type of flash.

df['camera_flash'].value_counts()

camera_flash
Yes, LED                        7456
Yes, Dual LED                   1737
Yes                              635
No                               205
Yes, Quad-LED                     68
Yes, Quad LED                     42
Yes, Triple LED                   25
Yes, Xenon                        12
Yes, True Tone Flash               8
Yes, Six LED Dual Tone Flash       5
Yes, 4 LED                         4
Yes, Quad LED Flash                3
Yes, Quad LED Ring                 3
Yes, Quad Flash                    3
Dual LED                           3
LED                                3
Yes, Yes                           2
Yes, Six-LED                       2
Yes, Penta LED                     1
Yes, 4 Rear Flashes                1
Yes, T-Flash                       1
Yes, Front and Rear Flash          1
Yes, Penta-LED                     1
Name: count, dtype: int64

In [54]:
has_flash = df['camera_flash'].fillna('').apply(lambda x: True if 'Yes' in x else False).astype('category')
df.insert(df.columns.get_loc('camera_flash')+1,'has_flash',has_flash)
df['has_flash'].value_counts()

has_flash
True     10010
False     5767
Name: count, dtype: int64

## camera_front_camera

In [55]:
# Issue - It has the camera megapixels and type of camera, and it also includes information about the flash types.

df['camera_front_camera'] = df['camera_front_camera'].str.replace('\u2009',' ')
df['camera_front_camera'].value_counts()

camera_front_camera
No                                                               6226
5 MP                                                             1180
0.3 MP                                                           1021
2 MP                                                              786
8 MP                                                              742
                                                                 ... 
Punch Hole 16 MP f/2.1                                              1
8 MP f/2 with Dual LED Flash                                        1
Punch Hole 60 MP f/2.2 (Wide Angle)\n60 MP f/2.2 (Wide Angle)       1
32 MP f/2.4 (Main)                                                  1
Punch Hole 8 MP 1.12µm (Wide Angle)                                 1
Name: count, Length: 1003, dtype: int64

In [56]:
front_camera_mp = df['camera_front_camera'].str.extract(r'([\d+.]+) MP')
has_front_camera = front_camera_mp.notna().astype('category')
df.insert(df.columns.get_loc('camera_front_camera')+1,'has_front_camera',has_front_camera)
df.insert(df.columns.get_loc('camera_front_camera')+2,'front_camera_mp',front_camera_mp)

## technical_cpu

In [57]:
# Issue - Most of the data contains a number of the cores of the CPU and its cycle speed, but some don't have core numbers.
# cycle speed has three units, KHz, GHz and MHz

df['technical_cpu'] = df['technical_cpu'].str.replace('\u2009',' ')

In [58]:
def convert_to_ghz(clock_speed):
    try:
        value, unit = clock_speed.split(" ")
        value = float(value)
        if unit == "GHz":
            return value
        elif unit == "MHz":
            return value / 1000
        elif unit == "KHz":
            return value / 1000000
    except Exception as e:
        return 0

In [59]:
df['technical_cpu'] = df['technical_cpu'].str.replace('\u2009',' ')
cpu_core_type = df['technical_cpu'].str.extract('([\S]+) Core')
clock_speed_ghz = df['technical_cpu'].str.extract('([\d.]+\s*[G|M|K]Hz)')[0].apply(convert_to_ghz)

df.insert(df.columns.get_loc('technical_cpu')+1,'cpu_core_type',cpu_core_type)
df.insert(df.columns.get_loc('technical_cpu')+2,'cpu_clock_speed_ghz',clock_speed_ghz)

## battery_type

In [60]:
#  No Issue
is_battery_removable = df['battery_type'].apply(lambda x: True if x=='Removable Battery' else False).astype('category')
df.insert(df.columns.get_loc('battery_type')+1,'is_battery_removable',is_battery_removable)

In [61]:
is_battery_removable.value_counts()

battery_type
True     8362
False    7415
Name: count, dtype: int64

## battery_size

In [62]:
# Issue - This field contains information about battery size in mAh and battery material. Some rows don't have battery material.
#         The battery type is written differently, like Li-ion Battery, Li-ion, Lion, Li-Po Battery, or Li-Polymer battery

df['battery_size'] = df['battery_size'].str.replace('\u2009',' ')

In [63]:
battery_size_mah = df['battery_size'].str.extract(r'([\d]+) mAh').astype('float')
df.insert(df.columns.get_loc('battery_size')+1,'battery_size_mah',battery_size_mah)

In [64]:
mapping = {
    "Li-ion Battery": "Li-Ion",
    "Li-ion": "Li-Ion",
    "lion": "Li-Ion",
    "Li-lon": "Li-Ion",
    "Li-on": "Li-Ion",
    "Lithium-ion Polymer": "Li-Polymer",
    "Lithium-Ion Polymer": "Li-Polymer",
    "Li-Po Battery": "Li-Polymer",
    "Li-Polymer": "Li-Polymer",
    "Lithium": "Lithium",
    "Lithium Ion": "Lithium",
    "Lithlum Battery": "Lithium",
    "Graphite": "Graphite",
    "Polymer": "Polymer",
    "Ni-MH Battery": "Ni-MH",
    "1000": "Unknown",
    "NA": "Unknown"
}

battery_material = df['battery_size'].str.split(',').str.get(1).str.strip().replace(mapping).astype('category')
df.insert(df.columns.get_loc('battery_size')+2,'battery_material',battery_material)

## design_dimensions

In [65]:
# Issue - Most of the data has dimensions in mm, but a few also have inches.

df['design_dimensions'].value_counts()

design_dimensions
-                           96
47 x 112 x 10 mm            41
75.9 x 164.2 x 9.1 mm       18
52 x 127 x 14.2 mm          16
25.5 x 80 x 50 mm           15
                            ..
141.5 x 70.7 x 11 mm         1
75.2 x 153.6 x 7.9 mm        1
69.59 x 139.69 x 9.69 mm     1
66.3 x 131.69 x 8.8 mm       1
45.8 x 125 x 10.1 mm         1
Name: count, Length: 6301, dtype: int64

In [66]:
dim = df['design_dimensions'].str.extract(r'([\d.]+) x ([\d.]+) x ([\d.]+) mm').apply(pd.to_numeric)
# Converting to inch
hight_inch =  dim[0] * 0.0393701
width_inch =  dim[1] * 0.0393701
depth_inch =  dim[2] * 0.0393701

df.insert(df.columns.get_loc('design_dimensions')+1,'hight_inch',hight_inch)
df.insert(df.columns.get_loc('design_dimensions')+2,'width_inch',width_inch)
df.insert(df.columns.get_loc('design_dimensions')+3,'depth_inch',depth_inch)

## design_weight

In [67]:
#  No Issue.
weight_gram = df['design_weight'].str.extract(r'([\d]+)').astype('float')
# weight_gram
df.insert(df.columns.get_loc('design_weight')+3,'weight_gram',weight_gram)

## display_ppi

In [68]:
# No Issue

df['display_ppi'] = df['display_ppi'].str.extract('([\d.]+)').astype('float')

## connectivity_volte

In [69]:
# Issue - It has two types of data: 1. volte-connectivity and type of connecctivity

df['connectivity_volte'].value_counts()

connectivity_volte
Yes                   3823
Yes, Dual Stand-By    2394
Dual Stand-By            3
Name: count, dtype: int64

In [70]:
has_volte = df['connectivity_volte'].apply(lambda x: False if pd.isna(x) else True).astype('category')
has_volte.value_counts()
df.insert(df.columns.get_loc('has_4g'),'has_volte',has_volte)

## technical_chipset

In [71]:
# Issue - It combines the manufacturer, series and model of the chipset.
#         Sometimes, only manufacturers and sometimes only models are available in the data
def extract_chipset_info(chipset):
    parts = [part.capitalize().strip() for part in chipset.split()]
    manufacturer_list = ['intel','nvidia','broadcom','apple','spreadtrum']

    if len(parts) == 0:
        return
    if len(parts) == 1 and parts[0].lower() in manufacturer_list: # e.g., 'Apple'
        return parts[0], None, None
    elif len(parts) == 1:
        return None, None, parts[0]
    elif len(parts) == 2:  # e.g., 'Unisoc T606'
        return parts[0], parts[1], None
    else:  # e.g., 'MediaTek Helio MT6739'
        return parts[0], parts[1], ' '.join(parts[2:])

In [72]:
chipset = df['technical_chipset'].fillna('').apply(lambda x: pd.Series(extract_chipset_info(x)))
chipset_manufacturer = chipset[0]
chipset_series = chipset[1]
chipset_model = chipset[2]

df.insert(df.columns.get_loc('technical_chipset')+1,'chipset_manufacturer',chipset[0])
df.insert(df.columns.get_loc('technical_chipset')+2,'chipset_series',chipset[1])
df.insert(df.columns.get_loc('technical_chipset')+3,'chipset_model',chipset[2])

## connectivity_5g

In [73]:
# No Issue
df['connectivity_5g'].value_counts()

connectivity_5g
Yes    2214
No      260
Name: count, dtype: int64

In [74]:
has_5g = df['connectivity_5g'].apply(lambda x: True if x=='Yes' else False).astype('category')
has_5g.value_counts()
df.insert(df.columns.get_loc('connectivity_5g'),'has_5g',has_5g)

## extra_water_resistance

In [75]:
# Issue - It has ip_rating, and water resistance time combined with the data, and sometimes only ip_rating are in it.
df['extra_water_resistance'].value_counts().sample(3)

extra_water_resistance
Yes                                               277
Water & Dust Proof Protection                       2
Yes, 1.5 m upto 30 min, Water-repellent Design      9
Name: count, dtype: int64

In [76]:
has_water_resistance = df['extra_water_resistance'].fillna('').apply(lambda x: True if 'Yes' in x else False).astype('category')
has_water_resistance
df.insert(df.columns.get_loc('extra_water_resistance')+1,'has_water_resistance',has_water_resistance)

## battery_wireless_charging

In [77]:
has_wireless_charging = df['battery_wireless_charging'].fillna('').apply(lambda x: True if 'Yes' in x else False).astype('category')
# has_wireless_charging.value_counts()
df.insert(df.columns.get_loc('battery_wireless_charging')+1,'has_wireless_charging',has_wireless_charging)

## battery_reverse_wireless_charging

In [78]:
df['battery_reverse_charging'].value_counts()

battery_reverse_charging
Yes                               537
Yes, 5W                           116
Yes, 10W                           89
Yes, 4.5W                          44
Yes, 2.5W                          12
Yes, 9W Reverse Charging           10
Yes, 9W                             9
Yes, 5W Reverse Charging            7
Yes, 7.5W                           4
Yes, 6W                             3
Yes, 18W                            2
10W Fast Charging                   1
Yes, 10W Fast Charging              1
Yes, 10 W                           1
Yes, 33W fast Charging              1
Yes, 15W                            1
Yes, 22.5W                          1
Yes, 10W Reverse Charging           1
Yes, Reverse wireless charging      1
Yes, 33W                            1
Name: count, dtype: int64

In [79]:
has_reverse_charging = df['battery_reverse_charging'].str.replace('10W Fast Charging','Yes').fillna('').apply(lambda x: True if 'Yes' in x else False)
has_reverse_charging_1 = df['battery_reverse_wireless_charging'].fillna('').apply(lambda x: True if 'Yes' in x else False)
combined_reverse_charging = (has_reverse_charging + has_reverse_charging_1).astype('category')
df.insert(df.columns.get_loc('battery_reverse_wireless_charging')+1,'has_reverse_charging',combined_reverse_charging)
# combined_reverse_charging.value_counts()